In [2]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will use a toy dataset from sklearn.

In [3]:
from sklearn import datasets

#Load dataset
sklearn_dataset = datasets.load_breast_cancer()
Xx  = sklearn_dataset.data
y = sklearn_dataset.target * 2 - 1    # labels must be in {-1, 1} for the hinge loss
X = np.ones((Xx.shape[0], Xx.shape[1] + 1 ))    # add a column of ones for intercept
X[:, :-1] = Xx
print("(N, D) =", X.shape)

(N, D) = (569, 31)


In [4]:
y.shape

(569,)

In [5]:
X.shape

(569, 31)

In [6]:
y.dot(X).shape

(31,)

In [7]:
w = np.ones(31)*-1

In [8]:
w.shape

(31,)

In [9]:
sum(w**2)

31.0

In [10]:
n = X.shape[0]
np.maximum(np.zeros(n),1 - (y * X.dot(w))).shape

(569,)

In [11]:
X.dot(w).shape

(569,)

## Prepare cost and prediction functions

In [12]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    n = X.shape[0]
    
    hinge = np.maximum(np.zeros(n),1 - (y * X.dot(w)))
    t1 = np.sum(hinge)
    reg = (lambda_/2) * np.sum(w ** 2)
    
    return t1 + reg

In [13]:
lambda_ = .004
calculate_primal_objective(y, X, w, lambda_)

457615.2179296

In [14]:
def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    p = X.dot(w)
    
    predict = np.ones(X.shape[0])
    predict[np.where(p < 0)] = -1
    
    correct = sum((y == predict))
    total = y.shape[0]
    
    return correct / total
    
    

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [15]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    x_n, y_n = X[n], y[n]
    
    def is_support(x_n, y_n, w):
        """a datapoint is support if max{} is not 0. """
        return y_n * x_n @ w < 1
    
    t1 = -y_n * x_n.T if is_support(x_n, y_n, w) else np.zeros_like(x_n.T)
    reg = lambda_ * w
    
    return num_examples * np.squeeze(t1) + reg

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [16]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 2 * int(1e5)
    gamma = 1e-4
    lambda_ = int(1e4)   # big because scales with N due to the formulation of the problem (not an averaged loss)
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        
        if it % 20000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0, cost=31780875.80463049
iteration=20000, cost=205.70035316871352
iteration=40000, cost=150.61567294835243
iteration=60000, cost=211.54563712081142
iteration=80000, cost=129.465700787664
iteration=100000, cost=130.83294497217952
iteration=120000, cost=127.86716039108039
iteration=140000, cost=125.42689697371519
iteration=160000, cost=129.50358166104408
iteration=180000, cost=174.77187759220007
training accuracy = 0.9279437609841827


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [20]:
X[1].dot(X[1])

5634504.79228238

Alec Notes:
- One thing they did that was smart was to check if g != 0 because if g==0 there won't be any update and you can just skip all the processing entirely
- I was overwriting W completely rather than updating it += vs = . Need to make sure I am understanding what needs to happen with each line fo code. 
- W I have the code completely wrong. I was looking in my notes for how to take the update to alpha and push it through to the primal vector but I don't think we went over it
    - The formula is to take the difference between the new and old alpha * y coordinate * x coordinate * 1/ lambda
    - You are updating every w at once

In [75]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    
#     t2 = lambda_ / (x_n @ x_n)
#     t3 = (1 - y_n * x_n @ w)
#     gradient = old_alpha_n + t2 * t3
    
#     alpha_n = np.clip(gradient, 0, 1)
#     alpha[n] = alpha_n
#     w = (1/lambda_)*(X.T@np.diag(y)@alpha)
    
        
    g = (1 - y_n * x_n.dot(w))

    if g != 0:
        alpha[n] = min(
            max(old_alpha_n + lambda_ * g / (x_n.T.dot(x_n)), 0.0),
            1.0)

        # compute the corresponding update on the primal vector w
        w += 1.0 / lambda_ * (alpha[n] - old_alpha_n) * y_n * x_n
    return w, alpha

Alec Notes
- There was a trick here to understanding that you didn't need to use the broken apart objective function and could instead use tricks like w = 1/lambda * X * Y * alpha
- Also alpha @ vector of ones is the same as just summing alpha which is probably faster


In [76]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
#     one = np.ones(X.shape[0])
    
#     t1 = alpha@one
#     t2 = (1/(2*lambda_))
#     t3 = alpha.T@np.diag(y)@X@X.T@np.diag(y)@alpha
    
#     return t1-t2*t3
    return np.sum(alpha)  - lambda_ / 2.0 * np.sum(w ** 2) # w = 1/lambda * X * Y * alpha

In [77]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 2*int(1e5)
    lambda_ = int(1e4)   # use same lambda as before in order to compare

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        #print(alpha)
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:683.80073, dual:0.00537, gap:683.79536
iteration=10000, primal:825.84152, dual:30.38351, gap:795.45800
iteration=20000, primal:144.26897, dual:56.14139, gap:88.12758
iteration=30000, primal:133.64952, dual:74.42565, gap:59.22387
iteration=40000, primal:146.57151, dual:88.35874, gap:58.21276
iteration=50000, primal:159.55940, dual:97.63074, gap:61.92866
iteration=60000, primal:147.89284, dual:103.51555, gap:44.37728
iteration=70000, primal:125.07226, dual:108.01160, gap:17.06066
iteration=80000, primal:129.06996, dual:111.13645, gap:17.93351
iteration=90000, primal:184.04196, dual:113.81011, gap:70.23184
iteration=100000, primal:124.07423, dual:115.73435, gap:8.33988
iteration=110000, primal:123.86403, dual:117.10434, gap:6.75968
iteration=120000, primal:123.45075, dual:118.09680, gap:5.35395
iteration=130000, primal:145.95619, dual:119.03609, gap:26.92010
iteration=140000, primal:134.08388, dual:119.71636, gap:14.36752
iteration=150000, primal:123.08827, dual:120.21

In [53]:
X.T@y

array([ 6.34189000e+02,  1.81533000e+03,  3.41546000e+03, -4.21997000e+04,
        1.12000400e+01, -2.18960000e+00, -1.76416693e+01, -9.47276600e+00,
        2.12877000e+01,  9.15548000e+00, -2.77081000e+01,  1.78961800e+02,
       -2.02558300e+02, -7.86130200e+03,  1.13155700e+00,  8.09847000e-01,
        4.14144600e-01,  3.26362000e-01,  3.00827000e+00,  4.36840300e-01,
        2.96009000e+02,  2.17942000e+03,  1.09061000e+03, -1.01997600e+05,
        1.39033500e+01, -1.42486100e+01, -3.61815130e+01, -1.20576790e+01,
        2.79026000e+01,  8.95647000e+00,  1.45000000e+02])

In [54]:
X.shape

(569, 31)

In [55]:
y.shape

(569,)